In [1]:
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import math
import sys
import random
from nltk.corpus import stopwords

M =  full with stop words  error rate: 0.21892071952

M =  full without stop words  error rate: 0.199200532978

M =  20000 random error rate: 0.30726182545

M =  10000 random error rate: 0.409593604264

M =  5000  error rate: 0.564956695536

### Top and log(1+f)

M =  20000  error rate: 0.20266489007328448

M =  10000  error rate: 0.220386409061 

M =  5000  error rate: 0.25076615589606926

### Random without removing stop words

M = 5000 mean 0.543730846102 std 0.0129676478143

In [2]:
directory = "/Users/liuxiyun/Dropbox/Study/16fall/CSE250B/Hw2/20news-bydate_parse/matlab/"

In [3]:
def readData(name):
    with open(directory + name) as f:
        content = [[int(num) for num in x.strip('\n').split(" ")] for x in f.readlines()]
    print len(content),
    return content
def readLabel(name):
    with open(directory + name) as f:
        content = [int(num) for num in f.readlines()]
    print len(content),
    return content
def readMap(name):
    d = {}
    with open(directory + name) as f:
        for line in f:
           (val, key) = line.split()
           d[int(key)] = val
    return d

In [4]:
trainData = readData('train.data')
trainLabel = readLabel('train.label')
testData = readData('test.data')
testLabel = readLabel('test.label')
trainMap = readMap('train.map')
testMap = readMap('test.map')

1467345 11269 967874 7505


In [5]:
with open("/Users/liuxiyun/Dropbox/Study/16fall/CSE250B/Hw2/vocabulary.txt") as f:
    vocabulary = [x.strip('\n') for x in f.readlines()]

In [6]:
numDoc = len(trainLabel)
numVoc = len(vocabulary)
numClass = 20
numTest = len(testLabel)
print numDoc,numVoc,numClass,numTest

11269 61188 20 7505


In [38]:
trainX = np.zeros((numDoc, numVoc))
testX = np.zeros((len(testLabel), numVoc))
for wordInfo in trainData:
    trainX[wordInfo[0]-1][wordInfo[1]-1] = wordInfo[2]
for wordInfo in testData:
    testX[wordInfo[0]-1][wordInfo[1]-1] = wordInfo[2]

In [7]:
rawIdxList = [num for num in range(0,numVoc)]

In [8]:
pi = [trainLabel.count(label)*1.0/numDoc for label in range(1,21)]

In [13]:
# Get stop words' index 
stopWords = [str(x) for x in stopwords.words("english")]
stopWordsIdx = []
for stopWords in stopWords:
    if stopWords in set(vocabulary):
        stopWordsIdx.append(vocabulary.index(stopWords))

In [85]:
# Pjw
eachClassWordCount = np.ones((20,numVoc)) # 1 for smoothing
for i in range(0,len(trainData)):
    docId = trainData[i][0]
    wordId = trainData[i][1]
    wordCount = trainData[i][2]
    docLabel = trainLabel[docId-1]
    eachClassWordCount[docLabel-1][wordId-1] += wordCount

numTotalWordCount = zip(np.sum(eachClassWordCount, axis = 0),rawIdxList)
numTotalWordCount.sort(key = lambda x:x[0],reverse = True)
topCountWordIdx = map(lambda x:x[1], numTotalWordCount)
topCountWordIdxRemoveStop = filter(lambda x: x not in set(stopWordsIdx),topCountWordIdx)

In [117]:
def predictThisDoc(pjw,docWordCount):
    maxProb = -sys.maxint - 1
    guessClass = 0
    logPi = np.log(pi)
    logPjw = np.log(pjw)
    for groupId in range(0, numClass):
        p = logPi[groupId]
        for word in range(0,len(docWordCount)):
            p += docWordCount[word]*logPjw[groupId][word]
        if p > maxProb:
            maxProb = p
            guessClass = groupId+1
    return guessClass
def predictThisDocNotLog(pjw,docWordCount):
    maxProb = -sys.maxint - 1
    guessClass = 0
    for groupId in range(0, numClass):
        p = pi[groupId]
        for word in range(0,len(docWordCount)):
            p += docWordCount[word]*math.log(pjw[groupId][word])
        if p > maxProb:
            maxProb = p
            guessClass = groupId
    return guessClass

In [167]:
def generatePJWAndTestX(M, eachClassWordCount, testX):
#     randIdx = randomSubVocab(M)
#     idxRandomNeedRemove  = filter(lambda x: x not in set(randIdx), rawIdxList)
#     idxNeedRemove = list(set(idxRandomNeedRemove + stopWordsIdx))

    idxNeedRemove = list(set( topCountWordIdxRemoveStop[M:]+ stopWordsIdx))
    filteredEachClassWordCount = np.delete(eachClassWordCount, idxNeedRemove, axis = 1)
    filteredTestX = np.delete(testX, idxNeedRemove, axis = 1)
    filteredEachClassWordCountLog = np.log(1+filteredEachClassWordCount)
    numWordInClass = np.sum(filteredEachClassWordCountLog, axis = 1)
    pjw = filteredEachClassWordCount/numWordInClass[:,None]
    

    return pjw, filteredTestX
def generatePJWAndTestXNotLog(M, eachClassWordCount, testX):
    idxNeedRemove = list(set( topCountWordIdxRemoveStop[M:]+ stopWordsIdx))
    filteredEachClassWordCount = np.delete(eachClassWordCount, idxNeedRemove, axis = 1)
    filteredTestX = np.delete(testX, idxNeedRemove, axis = 1)
    numWordInClass = np.sum(filteredEachClassWordCount, axis = 1)
    pjw = filteredEachClassWordCount/numWordInClass[:,None]

    return pjw, filteredTestX

In [128]:
numWordInClass = np.sum(eachClassWordCount, axis = 1)
pjw = eachClassWordCount/numWordInClass[:,None]

In [168]:
def test(M):
    filteredPJW, filteredTestX = generatePJWAndTestXNotLog(M, eachClassWordCount, testX)
    print len(filteredPJW[1])
    error= 0
    i = 0
    for i in range(len(filteredTestX)):
        predictLabel = predictThisDoc(filteredPJW, filteredTestX[i])
        if predictLabel!= testLabel[i]:
            error += 1
    error_rate = error* 1.0 / len(filteredTestX)
    print "M =", M," error_rate:" + str(error_rate)
    return error_rate

In [169]:
test(5000)

5000
M = 5000  error_rate:0.250766155896


0.25076615589606926

In [139]:
test(10000)

10000
M = 10000  error_rate:0.220386409061


0.22038640906062626

In [140]:
test(20000)

20000
M = 20000  error_rate:0.202664890073


0.20266489007328448

In [131]:
def randomSubVocab(M):
    randomIdx = random.sample(range(0, numVoc), M)
    return randomIdx

In [183]:
# Full vocab
test(numVoc)

M = 61188  error_rate:0.21894989339


0.21894989339019189

In [132]:
def getNewX(removedIdx, train = trainX, test = testX):
    newTrainX = np.delete(train, removedIdx, axis = 1)
    newTestX = np.delete(test, removedIdx, axis = 1)
    return newTrainX, newTestX

def randomSubVocIdx(M, fullIdx):
    randomIdx = random.sample(range(0, len(fullIdx)), M)
    subsetIdxList = [fullIdx[i] for i in randomIdx]
    return subsetIdxList

In [190]:
def testUsingLib(trainX,testX,M):
    clf = MultinomialNB()
    clf.fit(trainX,np.array(trainLabel))#Target values.
    MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
    predictLabel = clf.predict(testX)
    errorRate = len(filter(lambda x: x[0] != x[1],zip(predictLabel,testLabel)))*1.0/numTest
    print "M = ", M, " error rate:",errorRate,"this is me"
    return errorRate

In [230]:
testUsingLib(trainX,testX,'full with stop words')

M =  full with stop words  error rate: 0.21892071952


(array([ 1,  1,  1, ...,  1, 16,  2]), 0.21892071952031977)

In [232]:
newTrainX, newTestX = getNewX(stopWordsIdx)
testUsingLib(newTrainX,newTestX, 'full without stop words')

M =  full without stop words  error rate: 0.199200532978


(array([ 1,  1,  1, ...,  1, 16,  2]), 0.19920053297801465)

In [193]:
def randomSelectSubsetTest(M, trainX = trainX,testX = testX):
    idxWithoutStopWords = filter(lambda x: x not in stopWordsIdx, [i for i in range(0,numVoc)])
    randomIdx = set(randomSubVocIdx(M, [i for i in range(0,numVoc)]))
    idxNeedRemove = filter(lambda x: x not in randomIdx, [i for i in range(0,numVoc)])
    newTrainX, newTestX = getNewX(idxNeedRemove,trainX,testX)
    return testUsingLib(newTrainX,newTestX, M)

def selectTopWordsTest(M, trainX = trainX,testX = testX):
    idxNeedRemove = list(set(topCountWordIdxRemoveStop[M:]+ stopWordsIdx))
    newTrainX, newTestX = getNewX(idxNeedRemove,trainX,testX)
    return testUsingLib(newTrainX,newTestX, M)

In [136]:
randomTwentyThousand = []
for i in range(0,10):
    randomTwentyThousand.append(randomSelectSubsetTest(20000))

M =  20000  error rate: 0.318321119254
M =  20000  error rate: 0.341505662891
M =  20000  error rate: 0.337375083278
M =  20000  error rate: 0.328181212525
M =  20000  error rate: 0.32871419054
M =  20000  error rate: 0.338840772818
M =  20000  error rate: 0.330446369087
M =  20000  error rate: 0.322318454364
M =  20000  error rate: 0.319920053298
M =  20000  error rate: 0.327514990007


In [137]:
print np.mean(randomTwentyThousand)
print np.std(randomTwentyThousand)

0.329313790806
0.00754033224474


In [138]:
randomTenThousand = []
for i in range(0,10):
    randomTenThousand.append(randomSelectSubsetTest(10000))
print np.mean(randomTenThousand)
print np.std(randomTenThousand)

M =  10000  error rate: 0.44836775483
M =  10000  error rate: 0.440772818121
M =  10000  error rate: 0.436109260493
M =  10000  error rate: 0.441838774151
M =  10000  error rate: 0.435842771486
M =  10000  error rate: 0.425183211193
M =  10000  error rate: 0.408261159227
M =  10000  error rate: 0.432511658894
M =  10000  error rate: 0.427981345769
M =  10000  error rate: 0.420919387075
0.431778814124
0.0110545168399


In [283]:
print randomTenThousand

[0.44836775483011326, 0.4407728181212525, 0.4361092604930047, 0.44183877415056627, 0.43584277148567624, 0.4251832111925383, 0.40826115922718187, 0.4325116588940706, 0.427981345769487, 0.42091938707528315]


In [328]:
randomFiveThousand = [0.5530979347,0.559360426382,0.520319786809,0.549500333111,0.518854097268,0.543904063957,0.552831445703,0.550566289141,0.547501665556,0.541372418388]
# for i in range(0,10):
#     randomFiveThousand.append(randomSelectSubsetTest(5000))

In [330]:
print np.mean(randomFiveThousand)
print np.std(randomFiveThousand)

0.543730846102
0.0129676478143


In [135]:
randomSelectSubsetTest(5000)

M =  5000  error rate: 0.546702198534


0.5467021985343105

In [284]:
pjwStd = []
pjwMean = []
for i in range(0,numVoc):
    pjwStd.append(np.std(pjw[:,i]))
    pjwMean.append(np.mean(pjw[:,i]))

In [279]:
stdWithIdx = zip(pjwStd,rawIdxList)
stdWithIdx.sort(key = lambda x: x[0])
stdWithIdx = filter(lambda x: x[1] not in set(stopWordsIdx),stdWithIdx)

In [280]:
def selectSubsetByStd(M):
    subsetIdx = map(lambda (std,idx): idx, stdWithIdx)[:M]
    idxNeedRemove = filter(lambda x: x not in subsetIdx, [i for i in range(0,numVoc)])
    newTrainX, newTestX = getNewX(idxNeedRemove)
    testUsingLib(newTrainX,newTestX, M)

In [282]:
selectSubsetByStd(20000)

M =  20000  error rate: 0.701132578281


In [286]:
np.max(pjwMean)

0.034637029928602091

In [297]:
logTrainX = np.log2(trainX+1)
logTestX = np.log2(testX+1)

In [303]:
randomSelectSubsetTest(20000,logTrainX,logTestX)

M =  20000  error rate: 0.301265822785


In [304]:
randomSelectSubsetTest(10000,logTrainX,logTestX)

M =  10000  error rate: 0.38654230513


In [305]:
randomSelectSubsetTest(5000,logTrainX,logTestX)

M =  5000  error rate: 0.532311792139


In [69]:
a = np.array([[1,2],[3,4]])
a[:1]

array([[1, 2]])

In [192]:
selectTopWordsTest(5000)

M =  15000  error rate: 0.206928714191 this is me


0.20692871419053965

In [186]:
selectTopWordsTest(10000)

M =  10000  error rate: 0.220386409061 this is me


0.22038640906062626

In [187]:
selectTopWordsTest(20000)

M =  20000  error rate: 0.202664890073 this is me


0.20266489007328448

In [194]:
a = np.array([[1,2],[3,4]])
np.sum(a,axis=0)

array([4, 6])

In [220]:
topOneThousand = topCountWordIdxRemoveStop[:1000]
subsetVoc = []
for idx in topOneThousand:
    subsetVoc.append(vocabulary[idx])
probOfWordAmongClasses = []

In [221]:
totalWordCountInEachClass = np.sum(eachClassWordCount,axis = 0)
probWordInClass = eachClassWordCount/totalWordCountInEachClass

In [222]:
wordBelongsClass = np.argmax(probWordInClass, axis=0)+1

In [265]:
import collections
classWord = collections.defaultdict(list)
for idx in topOneThousand:
    if len(classWord[wordBelongsClass[idx]])>=30:
        continue
    else:
        classWord[wordBelongsClass[idx]].append(vocabulary[idx])

In [280]:
print classWord[7
               ]

['new', 'please', 'interested', 'price', 'original', 'sell', 'includes', 'offer', 'sale', 'cover', 'included', 'ii', 'asking', 'condition', 'appears', 'excellent', 'manual', 'shipping']
